# Anomalies in Data

In [1]:
import pandas as pd
from siuba import *

import numpy as np

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

In [2]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/function_data.csv', low_memory=False).drop('Unnamed: 0', axis=1)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## Looking into MPOs

using this map to see where the agency is in realtion to nearest mpo https://gisdata-caltrans.opendata.arcgis.com/datasets/b3e0ef95520843ba8c1d3b9c0fa9a607_0/explore

In [3]:
print(len(df>>count(_.mpo)>>arrange(-_.n)))

31


* Accoring to the Caltrans GIS data, there are 18 mpos

In [4]:
#number of obligations under MPO
df>>count(_.mpo)>>arrange(-_.n)

,mpo,n
18,SCAG,4988
12,MTC,3236
13,NON-MPO,2592
14,SACOG,1865
2,CFCG,1404
8,KCOG,998
26,STANCOG,759
0,AMBAG,745
21,SJCG,515
29,TCAG,489


### Which MPOs have the least amount of obligations? What Agencies do they include?

In [5]:
df >> count(_.mpo)>>arrange(-_.n)>> filter(_.n==1)

,mpo,n
6,FSTIP,1
11,MNOLTC,1
15,SANDA,1
20,SHASTA,1


#### MPO SANDA

In [6]:
df>>filter(_.mpo =='SANDA')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
12767,HSIPLN,5426(008),Big Bear Lake,2013-12-09,169650.0,0.0,188924.0,Authorized,NaN,8,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,SANDA,2013-01-01


* maybe meant "SANDAG" but Big Bear Lake is in "SCAG" jurisdiction
* looked to excel download file and this there is anothe Big Bear Lake and SANDAG instance

In [7]:
df>>filter(_.agency.str.contains('Big Bear')) >> filter(_.mpo.str.contains("SANDA"))

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
12567,HSIPLN,5426(011),Big Bear Lake,2014-02-20,27000.0,0.0,30000.0,Authorized,NaN,8,SCAG015,"Big Bear Boulevard (sr 18) From 1,000 Ft. West...",Widen Shoulder For Two Way Left Turn Lane,1,SANDAG,2014-01-01
12767,HSIPLN,5426(008),Big Bear Lake,2013-12-09,169650.0,0.0,188924.0,Authorized,NaN,8,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,SANDA,2013-01-01


google maps show the `project_location` intersections in the jurisdiction of Big Bear and not in SANDAG

#### MPO SHASTA

In [8]:
df >>filter(_.mpo=='SHASTA')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10037,HSIP,5073(075),Orange,2014-12-30,196199.0,0.0,218000.0,Authorized,NaN,12,SCAG015,Intersection Of Tustin Street And Mayfair Avenue,Left-turn Signal Phasing,2,SHASTA,2014-01-01


In [9]:
df>> filter(_.agency.str.contains('Shasta')) >> count(_.mpo)

,mpo,n
0,MTC,1
1,SACOG,3
2,STPA,129


In [10]:
df>>filter(_.agency.str.contains('Shasta')) >> filter(_.mpo=="MTC")

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
19168,ACSTP,15A6(002),Shasta County,2021-05-18,0.0,19251.73,21746.0,Authorized,5921,2,NaN,Ca20-3 Zogg Fire Eo Repairs On Platina Road,Emergency Opening,1,MTC,2021-01-01


* in the project location column, the intersection is Tustin and Mayfair which is in Orange Couny, or SCAG. Agency is also Orange, yet the mpo is Shasta

#### MPO MNOLTC and Agencies under

In [11]:
df>> filter(_.mpo=='MNOLTC')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10820,RPSTP,5205(019),Rialto,2014-09-10,2000000.0,-2000000.0,0.0,Authorized,NaN,8,20040830,Former Pacific Electric Railroad Right-of-way ...,Extend Class 1 Bicycle Trail,2,MNOLTC,2014-01-01


In [12]:
#looking to see if what other MPOs Rialto is in, should be in SCAG
df>> filter(_.agency=='Rialto') >> count(_.mpo)

,mpo,n
0,KCOG,1
1,MNOLTC,1
2,SANDAG,1
3,SCAG,15


In [13]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='KCOG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10667,SRTS,5205(014),Rialto,2014-10-08,0.0,0.0,85287.07,Authorized,NaN,8,SCAG016,"Eucalyptus Avenue, King Street, 2nd Street, En...","Curb, Ramps, And In-pavement Warning Signals",3,KCOG,2014-01-01


In [14]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='SANDAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
12416,HSIP,5205(017),Rialto,2014-03-07,-89313.0,0.0,-99336.0,Authorized,NaN,8,SCAG015,Various Intersections Throughout The City Of R...,Pedestrian Countdown Heads At 67 Intersections,2,SANDAG,2014-01-01


#### FSTIP

In [15]:
df>>filter(_.mpo=='FSTIP')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
12011,CML,5160(023),Los Banos,2014-04-16,-16940.31,0.0,-29901.84,Authorized,NaN,10,CMAQ11-05,In City Of Los Banos,Construct/install Pedestrian/bike Improvement,4,FSTIP,2014-01-01


* FSTIP is the Federal Statewide Transportation Improvement Program. 

### Number of Unique Agencies in Each MPO

In [16]:

df >> group_by(_.mpo) >> summarize(n=_.agency.nunique()) >> arrange(-_.n) 

,mpo,n
18,SCAG,282
12,MTC,152
13,NON-MPO,134
26,STANCOG,89
22,SJCOG,71
16,SANDAG,70
9,MCAG,49
8,KCOG,45
14,SACOG,41
2,CFCG,24


In [17]:
df >> filter(_.mpo=="SCAG") >> count(_.agency) >> arrange(-_.n)

,agency,n
130,Los Angeles,490
131,Los Angeles County,407
196,Riverside County,103
262,Ventura County,102
205,San Bernardino County,94
...,...,...
257,Tustin,1
258,Twentynine Palm,1
260,Usda Forest Ser,1
273,Westminister,1


## Function for Agencies in each MPO

creating a function to look into each MPO. What agencies are listed under the MPO name? Are the projects located in that jurisdiction? 

In [18]:
@interact
def find_agencies(place=df.mpo.sort_values().unique().tolist()):
    mpos = df[df.mpo==place]
    display(Markdown(f"**Agencies in {place}**"))
    display(mpos >> count(_.agency) >> arrange(-_.n))
    pd.set_option('display.max_rows', 500)

interactive(children=(Dropdown(description='place', options=('AMBAG', 'BCAG', 'CFCG', 'COFCG', 'ER NONE', 'FCO…

In [19]:
#using Siuba filter, we can start to look at the agenies under different MPOs

In [20]:
df>>filter(_.agency=='Marin County')>> filter(_.mpo=='SCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
19568,ACSTP,40A0(087),Marin County,2021-06-16,0.0,676463.01,764107.59,Authorized,5953,4,NaN,Lucas Valley Rd At Mp 3.92,Lucas Valley Rd At Mp 3.92 Roadway Damage. Un...,2,SCAG,2021-01-01


* projet located in Lucas Valley, which is not in SCAG

In [21]:
df>>filter(_.agency=='Sonora')>> count(_.mpo)

,mpo,n
0,COFCG,1
1,NON-MPO,3
2,SCAG,2
3,STANCOG,1


In [22]:
df>>filter(_.agency=='Sonora')>> filter(_.mpo=='SCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
8341,CML,5010(006),Sonora,2015-07-23,0.0,40296.0,0.0,Authorized,NaN,10,TRANS-HUB,S. Washington St. And Stockton St/hwy 49 (see ...,"Const Ped,transit Facility And Traffic Improv",2,SCAG,2015-01-01
8637,CML,5010(008),Sonora,2015-07-08,205000.0,0.0,205000.0,Authorized,NaN,10,SONORA-PR,244 S. Shepherd St (apn 002-164-16)/nw Corner ...,"Parking Lot, Utilize As Park & Ride (tc)",1,SCAG,2015-01-01


In [23]:
df>>filter(_.agency=='Shasta County')>> filter(_.mpo=='MTC')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
19168,ACSTP,15A6(002),Shasta County,2021-05-18,0.0,19251.73,21746.0,Authorized,5921,2,NaN,Ca20-3 Zogg Fire Eo Repairs On Platina Road,Emergency Opening,1,MTC,2021-01-01


In [24]:
df>>filter(_.agency=='Marina')>> filter(_.mpo=='AMBAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
4828,RPSTP,5416(008),Marina,2017-02-28,-13679.6,0.0,-8226.94,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),7,AMBAG,2017-01-01
5486,RPSTP,5416(008),Marina,2016-11-01,55689.0,0.0,55689.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),6,AMBAG,2016-01-01
5557,RPSTP,5416(008),Marina,2016-07-27,-55689.0,0.0,-57389.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),5,AMBAG,2016-01-01
6782,HSIP,5416(010),Marina,2016-05-05,1300033.0,0.0,1811770.00,Authorized,5416,5,HSIP08,Del Monte And Beach Road Roundabout,Construct Roundabout,2,AMBAG,2016-01-01
9128,RPSTP,5416(011),Marina,2015-05-21,1650000.0,0.0,1650000.00,Authorized,5416,5,CM01MO,Imjin Road From 4-lane Section To Reservation ...,Widen Roadway From 2 To 4 Lanes (tc),1,AMBAG,2015-01-01
9262,RPSTP,5416(008),Marina,2015-05-05,2000000.0,-2000000.0,0.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),4,AMBAG,2015-01-01
12766,RPSTP,5416(008),Marina,2014-02-07,0.0,0.0,0.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),3,AMBAG,2014-01-01
12824,RPSTP,5416(008),Marina,2014-01-09,0.0,2000000.0,2003300.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),2,AMBAG,2014-01-01
14911,HSIP,5416(010),Marina,2019-10-08,-19.6,0.0,0.00,Authorized,5416,5,HSIP08,Del Monte And Beach Road Roundabout,Construct Roundabout,3,AMBAG,2019-01-01


In [25]:
df>>filter(_.agency=='Kern County (District 9)')>> filter(_.mpo=='BCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
2472,CML,5961(006),Kern County (District 9),2018-02-05,1327950.0,0.0,1500000.0,Authorized,NaN,9,KER160508,Backus Road From Tehachapi Willow Springs Road...,Surfacing Unpaved Shoulders,1,BCAG,2018-01-01


In [26]:
df>>filter(_.agency=='Santa Barbara County')>> filter(_.mpo=='BCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
293,ER,38F0(011),Santa Barbara County,2018-09-06,83500.0,0.0,83500.0,Authorized,5912,3,NaN,0.35 Miles West Of The Intersection Of Skyway ...,Remove Mbgr And Install Midwest Guardrail Syst...,3,BCAG,2018-01-01
1041,ER,38F0(011),Santa Barbara County,2018-07-05,15000.0,-15000.0,0.0,Authorized,5912,3,NaN,0.35 Miles West Of The Intersection Of Skyway ...,Remove Mbgr And Install Midwest Guardrail Syst...,2,BCAG,2018-01-01


In [27]:
df>>filter(_.agency=='Bishop')>> filter(_.mpo=='KCOG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10665,RPSTP,5125(021),Bishop,2014-09-23,4049.0,0.0,4575.0,Authorized,NaN,9,NaN,Intersection Of N. 3rd Street And E. Pine Stre...,"New 1,000 Foot Paved Bike Path",5,KCOG,2014-01-01


In [28]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='MCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10831,RPSTP,5953(685),Los Angeles,2014-09-05,378000.00,-378000.0,0.0,Authorized,NaN,7,LAF5514,Vermont Ave - From Manchester Blvd To El Segun...,Class Ii Bike Lane Construction(3 Miles),3,MCAG,2014-01-01
10874,BRLSZD,5006(337),Los Angeles,2014-08-20,58000.00,0.0,72500.0,Authorized,NaN,7,LA000800,"Glendale-hyperion Viaduct, Bridge # 53c1882",Seismic Retrofit,8,MCAG,2014-01-01
11404,HSIP,5006(551),Los Angeles,2014-07-01,0.00,0.0,-935.0,Authorized,NaN,7,SCAG015/HSIP2-,Roxford St And I-5 (the Golden State Freeway),Install New Traffic Signal,4,MCAG,2014-01-01
11671,CML,5006(647),Los Angeles,2014-06-10,-0.01,0.0,0.0,Authorized,NaN,7,LAF1844,"City Of La, Crenshaw Blvd & Martin Luther King...",To Promote Pedestrian Activity In Project Limits,3,MCAG,2014-01-01
11712,STPLZ,5953(564),Los Angeles,2014-04-28,767290.00,0.0,1039725.0,Authorized,NaN,7,LA000800,East Fork Road Over The North Fork Of San Gabr...,Seismic Retrofit,4,MCAG,2014-01-01
11831,BRLSZD,5953(566),Los Angeles,2013-08-28,89563.00,0.0,402986.0,Authorized,NaN,7,LA000800,"Alameda Street Over Compton Creek, Bridge # 53...",Bridge Seismic Retrofit,3,MCAG,2013-01-01


In [29]:
df>>filter(_.agency=='Los Angeles')>> count(_.mpo)

,mpo,n
0,ER NONE,1
1,KCOG,4
2,MCAG,6
3,NON-MPO,12
4,SANDAG,14
5,SCAG,490
6,SJCOG,4
7,STANCOG,3


In [30]:
df>>filter(_.agency=='Indio')>> filter(_.mpo=='MCTC')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
7984,CML,5275(022),Indio,2015-10-08,81189.0,0.0,109275.0,Authorized,NaN,8,RIV091208,Intersection Of Jackson Street And Market Stre...,Traffic Signals And Wireless Interconnect (tc),2,MCTC,2015-01-01


In [31]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='SJCOG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
11012,RPSTP,5006(716),Los Angeles,2014-08-04,669000.00,-669000.0,0.0,Authorized,NaN,7,LAF3850/LA11G2,Vermont Ave B/n Hollywood Bl & Los Angeles Cit...,"Landscape, Install 6 Fully Irrigated Medians, Etc",3,SJCOG,2014-01-01
11636,BHLS,5006(193),Los Angeles,2014-03-28,710400.00,0.0,888000.0,Authorized,NaN,7,LA000800,Soto St Bridge Over Valley Blvd & Uprr; Bridge...,Bridge Rehabilitation,9,SJCOG,2014-01-01
11692,STP,5006(469),Los Angeles,2014-06-10,-39022.58,0.0,0.0,Authorized,NaN,7,NaN,"East Downtown, La (see State Comments Screen)",Truck Access Improvement,7,SJCOG,2014-01-01
11801,HPLUL,5006(518),Los Angeles,2014-04-28,-127527.00,0.0,-159408.0,Authorized,NaN,7,LAE2147,Devonshire St And Woodley Ave,Lighting N/w San Fernando Valley,3,SJCOG,2014-01-01


### MPOs with all correct agencies

* AMBAG
* KCAG
* SBCG
* TMPO

### MPOs with some misplaced agencies

* BCAG
* MTC
* SCAG
* KGOC
* MNOLTC
* FCOG 
* KCOG
* MCAG
* SJCOG
* STANCOG
* MCTC
* SACOG
* SJCOG
* STANCOG 
* TCAG


### MPOs not defined / all incorrect  
* CFCG - has Fresno Counties. Fresno is part of FCOG
* COFCG - combination of multiple MPOs
* FCOG - includes Ventura and Fresno
* FSTIP - a Prefix rather than a MPO
* MNOLTC - one entry
* SANDA - one entry
* SDAG - maybe SANDAG
* SANDAG - mostly SCAG Agencies
* SHASTA - one agency not in Shasta County
* SJCG - most likely SJCOG
* SLAC - maybe SLOCOG
* STACOG - STANCOG
* STNCOG - STANCOG
* STPA - maybe SRTA


## Other

### Agnecy Name 'YRTS'

In [33]:
df>>filter(_.agency=='Yrts')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
8674,CML,6305(011),Yrts,2015-07-09,39283.0,0.0,39283.0,Authorized,NaN,10,YARTS07,"Merced, Mariposa, Mono Counties",Yarts Public Outreach And Marketing (tc),1,ER NONE,2015-01-01
10563,CML,6305(008),Yrts,2014-10-13,45717.0,0.0,45717.0,Authorized,NaN,10,YARTS07,"In The Counties, Merced, Mariposa And Mono",Public Outreach And Marketing For Yarts (tc),2,SCAG,2014-01-01
11162,CML,6305(010),Yrts,2014-07-25,85000.0,0.0,85000.0,Authorized,NaN,10,YARTS07,"Merced, Mariposa, Mono Counties",Yarts Public Outreach And Marketing (tc),1,SJCOG,2014-01-01
11630,CML,6305(007),Yrts,2014-06-19,0.0,0.0,0.0,Authorized,NaN,10,YARTS07,"Merced,mariposa,mono Counties",Public Outreach And Marketing,2,STANCOG,2014-01-01


* 'Yrts' is actually ['Yarts'](https://yarts.com/), or Yosemite Area Regional Transportation System 

## Non-MPO Agencies
Looking at 'Non-MPO', 'ER NONE' and 'NaN' (or null)

#### Is Null

In [34]:
df>>filter(_.mpo.isnull())

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
20116,HSIP,7504(001),Caltrans,NaN,351200.0,0.0,351200.0,prog code Z300,7504,11,NaN,FTA transfer,Guardrail Var. Locations,1,NaN,NaN


### is ER NONE

In [35]:
df>>filter(_.mpo=='ER NONE')>>count(_.agency)

,agency,n
0,California City,1
1,Camarillo,1
2,Chino Hills,1
3,Coachella,1
4,El Centro,1
5,Los Angeles,1
6,Mission Viejo,1
7,Palm Springs,1
8,Pico Rivera,1
9,Ridgecrest,2


In [36]:
df>>filter(_.mpo=='ER NONE')>>filter(_.agency=='Ridgecrest')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10180,STP,5385(043),Ridgecrest,2015-01-02,-8826.89,0.0,-82797.55,Authorized,NaN,9,KER050406,N. Downs Street From W. Ward Avenue To W. Inyo...,Pavement Rehabilitation,3,ER NONE,2015-01-01
10207,STP,5385(044),Ridgecrest,2014-12-31,-25490.80,0.0,-83405.35,Authorized,NaN,9,KER100405,N. Downs Street From W. Drummond Avenue To W. ...,Pavement Rehabilitation,3,ER NONE,2014-01-01


In [37]:
df>>filter(_.mpo=='ER NONE')>>count(_.dist)

,dist,n
0,7,5
1,8,3
2,9,3
3,10,1
4,11,2
5,12,2


In [38]:
df>>filter(_.mpo=='ER NONE')>>filter(_.dist==8)

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10174,RPMSTP,5282(018),Palm Springs,2014-12-09,0.0,0.0,676628.81,Authorized,NaN,8,NaN,Gene Autry Trail From Whitewater River Wash To...,Widen Bridge And Roadway,9,ER NONE,2014-01-01
10176,SRTS,5294(009),Coachella,2015-01-02,0.0,0.0,0.00,Authorized,NaN,8,SCAG016,14 Locations In The Vicinity Of Five Elementar...,"Bulb-outs, Curb Extensions, Flashing Beacons, ...",5,ER NONE,2015-01-01
10184,SRTS,5467(007),Chino Hills,2014-10-30,764130.0,0.0,914816.00,Authorized,NaN,8,SCAG016,"Country Club Drive, Williams Avenue, Esther St...","Curb And Gutter, Sidewalk, And Ada-curb Ramps",2,ER NONE,2014-01-01


#### Is Non-MPO

In [39]:
df>>filter(_.mpo =='NON-MPO') >> count(_.agency)>>arrange(-_.n)

,agency,n
42,Humboldt County,627
62,Mendocino County,238
72,Nevada County,143
51,Lake County,131
13,Caltrans,105
11,Calaveras County,103
109,Tehama County,85
112,Trinity County,80
81,Plumas County,68
115,Tuolumne County,62
